In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from collections import Counter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Check json file structure in train/test directory

In [ ]:
# Check data type
# All data in train/test directory are json files
train_path_lists = []
test_path_lists = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    if dirname == "/kaggle/input/coleridgeinitiative-show-us-the-data/train":
        for filename in filenames:
            train_path_lists.append(os.path.join(dirname, filename))
        print(f"length of all train data: {len(filenames)}")
        print(f"number of json file in train data : {len([i for i in filenames if i[-4:] == 'json'])}")
        
    if dirname == "/kaggle/input/coleridgeinitiative-show-us-the-data/test":
        for filename in filenames:
            test_path_lists.append(os.path.join(dirname, filename))
        print(f"length of all test data: {len(filenames)}")
        print(f"number of json file in test data : {len([i for i in filenames if i[-4:] == 'json'])}")


In [ ]:
# Pick up a json file sample
# Index0 stores abstract, index1 stores introduction
json_open = open('../input/coleridgeinitiative-show-us-the-data/train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json', 'r')
json_df = json.load(json_open)
print(f"index0 keys: {json_df[0].keys()}")
print()
print(json_df[0])
print()
print(f"index1 keys: {json_df[1].keys()}")
print()
print(json_df[1])

In [ ]:
# All json file are same structure?
# ⇨ No, some file have more indexes. 
# Train directory
index_num = []
section_title = []
section_title_value = []

for path in train_path_lists:
    with open(path, 'r') as json_open:
        json_df = json.load(json_open)
        index_num.append(len(json_df))
        for i in range(len(json_df)):
            for key,value in json_df[i].items():
                section_title.append(key)
                if key == "section_title":
                    section_title_value.append(value)
print(f"index_num: {Counter(index_num)}")
print()
print(f"section_title: {Counter(section_title)}")
print()
print(f"section_title_value: {Counter(section_title_value)}")

In [ ]:
# Test directory
index_num = []
section_title = []
section_title_value = []

for path in test_path_lists:
    with open(path, 'r') as json_open:
        json_df = json.load(json_open)
        index_num.append(len(json_df))
        for i in range(len(json_df)):
            for key,value in json_df[i].items():
                section_title.append(key)
                if key == "section_title":
                    section_title_value.append(value)
print(f"index_num: {Counter(index_num)}")
print()
print(f"section_title: {Counter(section_title)}")
print()
print(f"section_title_value: {Counter(section_title_value)}")

In [ ]:
# Check other json.file
with open("../input/coleridgeinitiative-show-us-the-data/train/0008656f-0ba2-4632-8602-3017b44c2e90.json", 'r') as json_open:
    json_df = json.load(json_open)
json_df

## Check train.csv

In [ ]:
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train_df

In [ ]:
train_df.describe()

In [ ]:
# Thre is no null in all columns
train_df.info()

In [ ]:
# Conbine train.csv and json file which have same id
# Why there are 2 columns with same Id?
train_df[train_df["Id"]=="d0fa7568-7d8e-4db9-870f-f9c6f668c17b"].value_counts

In [ ]:
section_titles = []
with open("../input/coleridgeinitiative-show-us-the-data/train/d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json", 'r') as json_open:
    json_df = json.load(json_open)
print(json_df[0])
print()
print(json_df[1])


In [ ]:
pd.read_json("../input/coleridgeinitiative-show-us-the-data/train/d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json")

In [ ]:
[value for df in json_df for key, value in df.items() if key == 'section_title']

# Transform json to csv

In [ ]:
# Generate the training publications dataframe
df_train_publications = pd.DataFrame()

for train_file in train_path_lists:
    file_data = pd.read_json(train_file)
    file_data.insert(0,'pub_id', train_file.split('/')[-1].split('.')[0])
    df_train_publications = pd.concat([df_train_publications, file_data])

df_train_publications.to_csv("df_train_publications.csv",index=False)

df_train_publications

In [ ]:
# Generate the testing publications dataframe
df_test_publications = pd.DataFrame()

for test_file in test_path_lists:
    file_data = pd.read_json(test_file)
    file_data.insert(0,'pub_id', test_file.split('/')[-1].split('.')[0])
    df_test_publications = pd.concat([df_test_publications, file_data])

df_test_publications.to_csv("df_test_publications.csv",index=False)

df_test_publications

In [ ]:
import re
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)

In [ ]:
submission_df

In [ ]:
df_test_publications[df_test_publications['pub_id'] == "2100032a-7c33-4bff-97ef-690822c43466"].text.str.cat(sep='\n').lower()

In [ ]:
# If dataset_title includes the text of the id, appended the dataset_title
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
datasets_titles = [x.lower() for x in set(train_df['dataset_title'].unique()).union(set(train_df['dataset_label'].unique()))]

labels = []
for index in submission_df.index:
    publication_text = df_test_publications[df_test_publications['pub_id'] == index].text.str.cat(sep='\n').lower()
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append('|'.join(label))

submission_df['PredictionString'] = labels

submission_df.to_csv('submission.csv')

submission_df